# LGA Mapping

## Created by: Rebecca Jan
## Last updated: 2021-03-27

Set up notebook

In [ ]:
import geopandas as gpd
from geopandas.tools import sjoin
import numpy as np
import pandas as pd
import shapely
from shapely.geometry import Point,  Polygon

Set path and load csv files, uncomment which .shp files you want to map

In [ ]:
path ='C:\\Users\\vicsxk6\\OneDrive - Department of Health and Human Services. Victoria\\Documents\\Python Scripts\\lga_mapping\\'
sitrep = pd.read_csv(path+"input\sitrep.csv", low_memory=False, sep =",")

VIC lookup sourced from - https://data.gov.au/dataset/ds-dga-bdf92691-c6fe-42b9-a0e2-a4cd716fa811/details

In [ ]:
vic_lga = gpd.read_file(path+"reference\\vic_lga_polygon_shp\VIC_LGA_POLYGON_SHP.shp")
lga_lookup = pd.read_csv(path+"reference\lga_lookup.csv", low_memory=False, sep = ",")

Assign points to .csv data

In [ ]:
geometry = [Point(xy) for xy in zip(sitrep["Longitude_yest"], sitrep["Latitude_yest"])]
geo_df = gpd.GeoDataFrame(sitrep, crs="EPSG:4326", geometry=geometry)

Run inner join to pull in .shp data based on point

In [ ]:
vic_lga = vic_lga.to_crs(crs="EPSG:4326")
pointInPolys = sjoin(geo_df, vic_lga, how="left", op="within")

Run this to standardise naming conventions and bring in region

In [ ]:
pointInPolys['LGA_NAME'] = pointInPolys['LGA_NAME'].map(lga_lookup.set_index('LOOKUP')['RETURN'])

In [ ]:
df = pointInPolys.merge(lga_lookup, left_on='LGA_NAME', right_on='RETURN', how='left')

Sanity check data

In [ ]:
#df.REGION.unique()
#df.info()

Drop columns as needed

In [ ]:
df.drop(df.columns[[22,23,24,25,26,27,29,30,31,32,33]], axis = 1, inplace = True)

Convert columns to upper and export to csv

In [ ]:
df.columns= df.columns.str.upper()
df.to_csv(path+"output\lga_mapped.csv", index=False)